<a href="https://colab.research.google.com/github/FrancescoTorella/progettoLabAI/blob/main/new_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 68.1 MB/s eta 0:00:00


In [2]:
import torch
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.functional as TF
import os
import rasterio
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import shutil
import matplotlib.pyplot as plt
import time
import random
from sklearn.metrics import precision_score, recall_score, f1_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
class MultispectralDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir   #directory delle immagini di train senza label
        self.mask_dir = mask_dir    #directory delle label
        self.transform = transform  #trasformazione
        self.images = os.listdir(image_dir) #lista di stringhe contenente tutti i nomi delle immagini senza label

    def __len__(self):
        return len(self.images)    #lunghezza della stringa che contiene tutti i path, quindi grandezza del dataset di train

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index]) #ricostruisce il persorso completo dell'immagine corrispondente all'indice fornito
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".tif", "_mask.tiff").replace("PS-RGBNIR","Buildings")) #ricostruisce il percorso completo della label corrispondente all'indice fornito, inoltre sostituisce .jpg con .gif, qui si assume che il nome della label sia uguale a quello dell'immagine con l'estensione eccetto, devo modificare questa riga in seguito
        with rasterio.open(img_path) as src:
            image = src.read().transpose((1, 2, 0)).astype(np.float32) #trasforma l'immagine da formato (C, H, W) a (H, W, C)
        #with rasterio.open(mask_path) as src:
            #mask = src.read().transpose((1,2,0)).astype(np.float32)
        with rasterio.open(mask_path) as src:
          mask = src.read().astype(np.float32)
        #mask = np.array(Image.open(mask_path).convert("L"), dtype=np.long)


        #print("ciao")
        #plt.imshow(mask)
        #plt.title(f"Mask 1:")
        #plt.axis('off')
        #plt.show()
        #apre l'immagine e la converte in scala di grigi, inoltre la trasforma in un array numpy di float32
        mask[mask == 255.0] = 1.0 #sostituisce tutti i valori 255 con 1, serve per la normalizzazione
        mask = mask.squeeze(0)
        #plt.imshow(mask)
        #plt.title(f"Mask 2:")
        #plt.axis('off')
        #plt.show()

        if self.transform is not None: #se è stata definita una trasformazione
            augmentations = self.transform(image=image, mask=mask) #applica le trasformazioni all'immagine e alla label, restituisce un dizionario con chiavi image e mask
            image = augmentations["image"] #estrai dal dizionario l'immagine e la assegna alla variabile image
            mask = augmentations["mask"] #estrai dal dizionario la label e la assegna alla variabile mask

        #plt.imshow(mask)
        #plt.title(f"Mask 3:")
        #plt.axis('off')
        #plt.show()

        return image, mask

In [5]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

In [6]:
#salva il checkpoint del modello in un file .pth.tar, state è un dizionario che contiene lo stato del modello e dell'ottimizzatore, filename è il nome del file in cui salvare il checkpoint
def save_checkpoint(state, filename="/content/drive/My Drive/progettoLabAi3/computations/saved/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

#carica il checkpoint del modello da un file .pth.tar, checkpoint è un dizionario che contiene lo stato del modello e dell'ottimizzatore, model è l'istanza del modello, checkpoint è un dizionario che contiene lo stato del modello e dell'ottimizzatore salvato in un file .pth.tar, model è l'istanza del modello in cui caricare il checkpoint
def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):

    #show_random_image_from_directory(train_maskdir)

    #crea un'istanza della classe MultispectralDataset, che rappresenta il dataset di addestramento, image_dir è la directory delle immagini di addestramento, mask_dir è la directory delle label di addestramento, transform è la trasformazione da applicare al dataset
    train_ds = MultispectralDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers, #numero di processi in background da utilizzare per il caricamento dei dati
        pin_memory=pin_memory, #se True, i dati verranno copiati in memoria GPU prima di essere passati al modello, nel mio caso è inutile settarlo a True perché non ho una GPU dedicata
        shuffle=True, #se True, i dati verranno mescolati prima di essere passati al modello in ogni epoca di addestramento
    )

    #crea un'istanza della classe MultispectralDataset, che rappresenta il dataset di validazione, image_dir è la directory delle immagini di validazione, mask_dir è la directory delle label di validazione, transform è la trasformazione da applicare al dataset
    val_ds = MultispectralDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    #stessa cosa di train_loader ma per il dataset di validazione, infatti passiamo val_ds come primo argomento, però shuffle=False perché non serve mescolare i dati di validazione
    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

def check_accuracy(loader, model, loss_fn, file, device="cuda"):
    model.eval()
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    precision = 0
    recall = 0
    f1 = 0
    val_loss = 0
    iteration = 1
    with torch.no_grad():
        for x, y in loader:
            print(f'Iteration number: {iteration}')
            iteration += 1
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)
            precision += precision_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten(), zero_division=1)
            recall += recall_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten(), zero_division=1)
            f1 += f1_score(y.cpu().numpy().flatten(), preds.cpu().numpy().flatten(), zero_division=1)
            val_loss += loss_fn(preds, y).item()

    val_loss /= len(loader)
    accuracy = num_correct / num_pixels * 100
    dice = dice_score / len(loader)
    precision_avg = precision / len(loader)
    recall_avg = recall / len(loader)
    f1_avg = f1 / len(loader)

    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Dice score: {dice}")
    print(f"Precision: {precision_avg}")
    print(f"Recall: {recall_avg}")
    print(f"F1 Score: {f1_avg}")

    # Salva i risultati su un file di testo
    with open(file, "a") as f:
        f.write(f"Validation Loss: {val_loss:.4f}\n")
        f.write(f"Accuracy: {accuracy:.2f}\n")
        f.write(f"Dice score: {dice}\n")
        f.write(f"Precision: {precision_avg}\n")
        f.write(f"Recall: {recall_avg}\n")
        f.write(f"F1 Score: {f1_avg}\n")
        f.write("\n")  # Riga vuota per separare le epoche

    model.train()


def save_predictions_as_imgs(loader, model, folder="/content/drive/My Drive/progettoLabAi3/computations/saved/images/",
                             device="cuda"):
    model.eval()  # Imposta il modello in modalità di valutazione
    print(f"Saving images to {folder}")
    iteration = 1
    for idx, (x, y) in enumerate(loader):
        print("Iteration: ", iteration)
        iteration += 1
        x = x.to(device=device)
        y = y.to(device=device).unsqueeze(1)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()

        # Concatenare le immagini (maschera originale e predizione) lungo l'asse verticale
        combined = torch.cat((y, preds), dim=2)

        # Salvare l'immagine combinata
        torchvision.utils.save_image(combined, f"{folder}/comparison_{idx}.png")

    model.train()  # Ripristina il modello in modalità di addestramento

In [7]:
def visualize_image_mask(image, mask, index):
    # Utilizza solo i primi tre canali (R, G, B)
    image_rgb = image[:, :, :3]

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(image_rgb)
    axs[0].set_title(f"Image {index}")
    axs[1].imshow(mask, cmap='gray')
    axs[1].set_title(f"Mask {index}")
    plt.show()

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [14]:
# Hyperparameters etc.
LEARNING_RATE = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 10
NUM_WORKERS = 4
IMAGE_HEIGHT = 300
IMAGE_WIDTH = 300
STEP_SIZE = 9
PIN_MEMORY = True
LOAD_MODEL = True
TRAIN_IMG_DIR = "/content/PS-RGBNIR/train_set"
TRAIN_MASK_DIR = "/content/mask/train_set"
VAL_IMG_DIR = "/content/PS-RGBNIR/val_set"
VAL_MASK_DIR = "/content/mask/val_set"
MODEL_DIRECTORY = "/content/drive/My Drive/progettoLabAi3/computations/saved/model4/"


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def train_fn(loader, model, optimizer, loss_fn, scaler,file):
    loop = tqdm(loader)  # Aggiunge una barra di avanzamento



    # Enumerate restituisce una tupla contenente un contatore e il valore restituito dall'iterabile
    for batch_idx, (data, targets) in enumerate(loop):

        #print(f'data.shape: {data.shape}, data.dtype: {data.dtype}')
        #print(f'targets.shape: {targets.shape}, targets.dtype: {targets.dtype}')
        data = data.to(device=DEVICE)  # Trasferisce i dati sulla GPU
        targets = targets.float().to(device=DEVICE)  # Trasferisce le label sulla GPU e aggiunge una dimensione

        with torch.cuda.amp.autocast():
            predictions = model(data).squeeze(1) #predictions è un tensore di predizioni, ad esempio (16, 1, 300, 300)
            loss = loss_fn(predictions, targets)



        # backward
        optimizer.zero_grad() #azzera i gradienti
        scaler.scale(loss).backward() #calcola i gradienti e scala i gradienti
        scaler.step(optimizer) #aggiorna i pesi
        scaler.update() #descala i pesi


        loop.set_postfix(loss=loss.item())
        last_loss = loss.item()  # Memorizza l'ultima loss calcolata

    # Salva l'ultima loss calcolata su un file di testo
    with open(file, "a") as f:
        f.write(f"Train Loss: {last_loss:.4f}\n")



def main():

    set_seed(42)

    print("Creating transforms")
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                #mean=[84.882515, 114.9725, 122.400505, 264.03476],
                #std=[112.35377, 132.05618, 154.0384, 294.54465],
                mean=[0.0,0.0,0.0,0.0],
                std=[1.0,1.0,1.0,1.0],
                #max_pixel_value=65535.0,
            ),
            ToTensorV2(),
        ],
    )
    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                #mean=[81.27881, 111.04898, 117.07683, 261.56107],
                #std=[110.35951, 129.69858, 151.043, 296.55923],
                mean=[0.0,0.0,0.0,0.0],
                std=[1.0,1.0,1.0,1.0],
                #max_pixel_value=65535.0,
            ),
            ToTensorV2(),
        ],
    )

    print("Creating model")
    model = UNET(in_channels=4, out_channels=1).to(DEVICE)
    print("Model created")
    print("creating loss function and optimizer")
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE)
    print("loss function and optimizer created")

    print("Creating data loaders")
    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )


    print("data loaders created")
    if LOAD_MODEL:
        load_checkpoint(torch.load(MODEL_DIRECTORY +"my_checkpoint.pth.tar"), model)

    print("checking accuracy on the validation set before training")
    #check_accuracy(val_loader, model, loss_fn, device=DEVICE)
    scaler = torch.cuda.amp.GradScaler()

    print("Starting training")
    for epoch in range(NUM_EPOCHS):
        print(f"epoch {epoch}")
        print("starting training")
        train_fn(train_loader, model, optimizer, loss_fn, scaler, file= MODEL_DIRECTORY +"outputs/outputs.txt")
        print("training done")

        print("saving model")
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint,filename = MODEL_DIRECTORY + "my_checkpoint.pth.tar")
        print("model saved")

        print("checking accuracy on the validation set after training")
        check_accuracy(val_loader, model, loss_fn,file= MODEL_DIRECTORY +"outputs/outputs.txt", device=DEVICE )

        if((epoch+1) % 10 == 0):
          print("saving predictions to folder")
          save_predictions_as_imgs(
              val_loader, model, folder=MODEL_DIRECTORY +"images/", device=DEVICE
          )

        scheduler.step()

if __name__ == "__main__":
    main()

Creating transforms
Creating model
Model created
creating loss function and optimizer
loss function and optimizer created
Creating data loaders
data loaders created
=> Loading checkpoint
checking accuracy on the validation set before training
Starting training
epoch 0
starting training


100%|██████████| 149/149 [00:54<00:00,  2.74it/s, loss=0.0831]


training done
saving model
=> Saving checkpoint
model saved
checking accuracy on the validation set after training
Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21
Iteration number: 22
Iteration number: 23
Iteration number: 24
Iteration number: 25
Iteration number: 26
Iteration number: 27
Iteration number: 28
Iteration number: 29
Iteration number: 30
Iteration number: 31
Iteration number: 32
Validation Loss: 0.6767
Accuracy: 97.46
Dice score: 0.8089284896850586
Precision: 0.8986424681131684
Recall: 0.7362578182345533
F1 Score: 0.8089285445989601
epoch 1
starting training


100%|██████████| 149/149 [00:54<00:00,  2.74it/s, loss=0.0796]


training done
saving model
=> Saving checkpoint
model saved
checking accuracy on the validation set after training
Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21
Iteration number: 22
Iteration number: 23
Iteration number: 24
Iteration number: 25
Iteration number: 26
Iteration number: 27
Iteration number: 28
Iteration number: 29
Iteration number: 30
Iteration number: 31
Iteration number: 32
Validation Loss: 0.6760
Accuracy: 97.69
Dice score: 0.8300994038581848
Precision: 0.8946429767750219
Recall: 0.7748459459930862
F1 Score: 0.8300994493882672
epoch 2
starting training


...

training done
saving model
=> Saving checkpoint
model saved
checking accuracy on the validation set after training
Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21
Iteration number: 22
Iteration number: 23
Iteration number: 24
Iteration number: 25
Iteration number: 26
Iteration number: 27
Iteration number: 28
Iteration number: 29
Iteration number: 30
Iteration number: 31
Iteration number: 32
Validation Loss: 0.6759
Accuracy: 97.84
Dice score: 0.8469679355621338
Precision: 0.8762148678106705
Recall: 0.8200549779029113
F1 Score: 0.8469678340668292
epoch 8
starting training


100%|██████████| 149/149 [00:54<00:00,  2.74it/s, loss=0.0213]


training done
saving model
=> Saving checkpoint
model saved
checking accuracy on the validation set after training
Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21
Iteration number: 22
Iteration number: 23
Iteration number: 24
Iteration number: 25
Iteration number: 26
Iteration number: 27
Iteration number: 28
Iteration number: 29
Iteration number: 30
Iteration number: 31
Iteration number: 32
Validation Loss: 0.6739
Accuracy: 98.19
Dice score: 0.8713122606277466
Precision: 0.9096989292042065
Recall: 0.836305799654519
F1 Score: 0.8713123597683735
epoch 9
starting training


100%|██████████| 149/149 [00:54<00:00,  2.75it/s, loss=0.0468]


training done
saving model
=> Saving checkpoint
model saved
checking accuracy on the validation set after training
Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
Iteration number: 21
Iteration number: 22
Iteration number: 23
Iteration number: 24
Iteration number: 25
Iteration number: 26
Iteration number: 27
Iteration number: 28
Iteration number: 29
Iteration number: 30
Iteration number: 31
Iteration number: 32
Validation Loss: 0.6736
Accuracy: 98.33
Dice score: 0.8834089040756226
Precision: 0.9005082836175586
Recall: 0.8671657231511516
F1 Score: 0.8834088641075217
saving predictions to folder
Saving images to /content/drive/My D